# Donwstream

# Imports

In [9]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict, load_from_disk
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling
)
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig
import torch
import wandb

from utils import tokenize_pretrain_dataset

# Configs

In [2]:
attempt = "attempt_5"

data_path = "../data/pubmed_baseline/"
train_data_path = data_path + "pubmed_train.csv"
val_data_path = data_path + "pubmed_val.csv"

max_len = 360

model_output_dir = f"../models/phi_pubmed_pretrained_{attempt}"

model_id = "microsoft/Phi-3.5-mini-instruct"

## Hyperparameters

In [3]:
lora_r = 16
lora_alpha = 32
lora_target_modules = ["qkv_proj", "o_proj"]
batch_size = 32
quantization = None
lora_dropout = 0.05
epochs = 3
learning_rate = 3e-5
warmup_steps = 600

# Dataset loading

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

In [5]:
val_df = pd.read_csv(val_data_path)
# train_df = pd.read_csv(train_data_path)

In [6]:
val_set = tokenize_pretrain_dataset(tokenizer, val_df, max_len)
# train_set = tokenize_pretrain_dataset(tokenizer, train_df.loc[:200_001, :], max_len)

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 1252.03 examples/s]


In [10]:
# train_set.save_to_disk(data_path + "tokenized_phi")
train_set = load_from_disk(data_path + "tokenized_phi")

# Setup

In [11]:
wandb.init(
    project="pubmed-pretrain",
    name=attempt,
    config={
        "model": model_id,
        "lora_r": lora_r,
        "lora_alpha": lora_alpha,
        "batch_size": batch_size,
        "epochs": epochs,
        "quantization": quantization,
        "lora_target_modules": lora_target_modules
    }
)

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hasindumadushan325 (hasindumadushan325-university-of-peradeniya) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Model

In [ ]:
## uncomment for qunatization
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=quantization=="4bit",
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.90it/s]


In [13]:
model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)

In [14]:
lora_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

# Train

In [15]:
training_args = TrainingArguments(
    output_dir=model_output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    eval_strategy="epoch",  # ✅ eval at each epoch
    save_strategy="epoch",
    logging_steps=50,
    learning_rate=learning_rate,
    warmup_steps=warmup_steps,
    fp16=True,
    report_to="wandb",
    run_name=attempt,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)


/tmp/ipykernel_87156/376439240.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

trainer.save_model(model_output_dir + "/final")
tokenizer.save_pretrained(model_output_dir + "/final")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
You are not running the flash-attention implementation, expect numerical differences.


Epoch,Training Loss,Validation Loss


### Merge final model

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

finetuned_model = PeftModel.from_pretrained(base_model, model_output_dir + "/final")
tokenizer = AutoTokenizer.from_pretrained(model_output_dir + "/final", trust_remote_code=True)
merged_model = finetuned_model.merge_and_unload()

In [ ]:
merged_model.save_pretrained(model_output_dir +  "/final_merged", safe_serialization=True)
tokenizer.save_pretrained(model_output_dir + "/final_merged")